In [13]:
import json, re

def loadFile(path: str):

    with open(path, 'r') as file:
        return json.loads(file.read())

In [14]:
htsfile = loadFile('../../db_hts/htsdata/htsdata.json')

In [15]:
countSup = 0
for record in htsfile:
    if record['superior'] == 'true':
        countSup += 1

print(f'Count of records: {len(htsfile)}\nCount of superior true records: {countSup}')

Count of records: 35744
Count of superior true records: 6001


In [27]:
empty_hts_review = []
review_index = 0
prev_EH = False

for index, record in enumerate(htsfile):

    if record['htsno'] == '' and prev_EH == False:
        
        previous_hts_record = htsfile[index-1]

        empty_hts_review.append({
            'recordIndexes': [index],
            'previous': previous_hts_record['htsno']
        })

        if previous_hts_record['indent'] == record['indent']:

            empty_hts_review[review_index]['EQUAL'] = True
        else:
            empty_hts_review[review_index]['EQUAL'] = False

        review_index += 1
        prev_EH = True

    elif record['htsno'] == '' and prev_EH == True:

        empty_hts_review[review_index-1]['recordIndexes'].append(index)

    elif record['htsno'] != '':

        prev_EH = False

empty_hts_review

[{'recordIndexes': [1], 'previous': '0101', 'EQUAL': False},
 {'recordIndexes': [13], 'previous': '0102', 'EQUAL': False},
 {'recordIndexes': [15], 'previous': '0102.21.00', 'EQUAL': False},
 {'recordIndexes': [18], 'previous': '0102.21.00.20', 'EQUAL': False},
 {'recordIndexes': [26], 'previous': '0102.29.40', 'EQUAL': False},
 {'recordIndexes': [29], 'previous': '0102.29.40.28', 'EQUAL': False},
 {'recordIndexes': [32], 'previous': '0102.29.40.38', 'EQUAL': False},
 {'recordIndexes': [35, 36], 'previous': '0102.29.40.58', 'EQUAL': False},
 {'recordIndexes': [41], 'previous': '0102.29.40.68', 'EQUAL': False},
 {'recordIndexes': [44], 'previous': '0102.29.40.74', 'EQUAL': False},
 {'recordIndexes': [47], 'previous': '0102.29.40.84', 'EQUAL': False},
 {'recordIndexes': [53, 54], 'previous': '0102.39.00.10', 'EQUAL': True},
 {'recordIndexes': [57], 'previous': '0102.39.00.28', 'EQUAL': False},
 {'recordIndexes': [60], 'previous': '0102.39.00.38', 'EQUAL': False},
 {'recordIndexes': [63],

In [26]:
countFollowEH = 0
for record in empty_hts_review:

    if len(record['recordIndexes']) > 1:
        countFollowEH += 1

countFollowEH

687